### Import Dependencies

In [8]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import add_to_shopping_cart, get_shopping_cart, remove_from_cart, get_formatted_items_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver
import psycopg2
from psycopg2.extras import RealDictCursor, execute_batch
import numpy as np

### Fictional Warehouses

In [9]:
warehouses = [
    {
        "warehouse_id": "DE-BER-01",
        "warehouse_location": "Berlin, Germany",
        "warehouse_name": "Berlin Distribution Center"
    },
    {
        "warehouse_id": "DE-MUN-01",
        "warehouse_location": "Munich, Germany",
        "warehouse_name": "Munich Logistics Hub"
    },
    {
        "warehouse_id": "DE-HAM-01",
        "warehouse_location": "Hamburg, Germany",
        "warehouse_name": "Hamburg North Warehouse"
    },
    {
        "warehouse_id": "FR-PAR-01",
        "warehouse_location": "Paris, France",
        "warehouse_name": "Paris Central Depot"
    },
    {
        "warehouse_id": "FR-LYO-01",
        "warehouse_location": "Lyon, France",
        "warehouse_name": "Lyon Regional Warehouse"
    },
    {
        "warehouse_id": "FR-MAR-01",
        "warehouse_location": "Marseille, France",
        "warehouse_name": "Marseille Mediterranean Hub"
    }
]

### Simulate Stock Availability for each of the warehouse

#### Retrieve all item IDs from Amazon items Qdrant Collection

In [10]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [11]:
dummy_vector = np.zeros(1536).tolist()

In [12]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [13]:
payload.points

[ScoredPoint(id=201, version=3, score=0.0, payload={'parent_asin': 'B09SBZZVL3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=429, version=3, score=0.0, payload={'parent_asin': 'B0B2JYLGNX'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=374, version=3, score=0.0, payload={'parent_asin': 'B09YCSP9FH'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=473, version=3, score=0.0, payload={'parent_asin': 'B0C1YLPFRK'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=407, version=3, score=0.0, payload={'parent_asin': 'B0C89R9NFZ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=97, version=3, score=0.0, payload={'parent_asin': 'B09YCXVRFN'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=82, version=3, score=0.0, payload={'parent_asin': 'B0BCGGV8R1'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=95, version=3, score=0.0, payload={'parent_asin': 'B0B2DNMM8Y'}, vector=None

In [14]:
parent_asin_list = [point.payload["parent_asin"] for point in payload.points]

In [15]:
parent_asin_list

['B09SBZZVL3',
 'B0B2JYLGNX',
 'B09YCSP9FH',
 'B0C1YLPFRK',
 'B0C89R9NFZ',
 'B09YCXVRFN',
 'B0BCGGV8R1',
 'B0B2DNMM8Y',
 'B0B96RSG2Y',
 'B09TZZR8DB',
 'B09SVN1YLY',
 'B0BKFPSBDQ',
 'B07MZRWKWM',
 'B0B9RXGWD5',
 'B0BMKH3KYC',
 'B0C7WKF7T6',
 'B0BVDGRLDZ',
 'B0BMLV9V7V',
 'B0B764M1MF',
 'B0B8NK3TPY',
 'B0BD9DVMCX',
 'B0BRY7T37H',
 'B09ZP22RN4',
 'B0B54FNTZ3',
 'B0BML3SQSM',
 'B0B6CHRTGJ',
 'B09XF1JXXB',
 'B0BVM12TMZ',
 'B0B8N2ZXX4',
 'B0BZ871HQC',
 'B0BC19YPZG',
 'B0BJKPTD7J',
 'B0BP1RZSF4',
 'B0BDLVB11Y',
 'B08CXFGV6B',
 'B0B8G12853',
 'B0BJ5ZCY2V',
 'B0BV6GY5LD',
 'B09QRR4C8Y',
 'B0BXNZY997',
 'B0BRSMHRS3',
 'B0BVB5WXGV',
 'B09TSRX7LD',
 'B0B57S52FS',
 'B0B3N3P3P1',
 'B09Y1FVYNW',
 'B0B3DCLY8H',
 'B0B15NNYSR',
 'B0C4FDPB28',
 'B0B5W8GLYG',
 'B0BRXLKBW1',
 'B0C77XMZ4K',
 'B0BGMCHZVL',
 'B0B5H3NW22',
 'B09MQJ1KZQ',
 'B0CG1C78YF',
 'B0B97LFT8Q',
 'B0BM5RCRHK',
 'B09TTVBJZV',
 'B0BBGMCDTP',
 'B09ZY72MQ1',
 'B0B9MZPGVD',
 'B0B2R74DGX',
 'B0BPBTWX5Z',
 'B0BWMJ53MD',
 'B0C4DLHNRD',
 'B0CCKQ9H

In [16]:
len(parent_asin_list)

1000

### Generate Synthetic availability for all items in Qdrant

In [17]:
def generate_inventory_data(warehouses, product_ids, availability_rate=0.75):
    
    inventory_records = []
    
    for warehouse in warehouses:
        for product_id in product_ids:
            # 75% chance the product is available in this warehouse
            if random.random() < availability_rate:
                total_quantity = random.randint(0, 100)
                
                # Only add to inventory if quantity > 0
                if total_quantity > 0:
                    inventory_records.append({
                        "warehouse_id": warehouse["warehouse_id"],
                        "warehouse_location": warehouse["warehouse_location"],
                        "warehouse_name": warehouse["warehouse_name"],
                        "product_id": product_id,
                        "total_quantity": total_quantity,
                        "reserved_quantity": 0  # Starting with no reservations
                    })
    
    return inventory_records

In [18]:
inventory_data = generate_inventory_data(warehouses, parent_asin_list, availability_rate=0.75)

In [19]:
inventory_data

[{'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0B2JYLGNX',
  'total_quantity': 11,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B09YCSP9FH',
  'total_quantity': 29,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0C1YLPFRK',
  'total_quantity': 85,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0B2DNMM8Y',
  'total_quantity': 45,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0B96RSG2Y',
  'total_quantity': 50,
  

In [20]:
1000*0.75*6

4500.0

In [21]:
len(inventory_data)

4457

### Write syntehetic data to Postgres

In [22]:
def insert_inventory_to_db(inventory_records):
   
    try:
        # Connect to the database
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            database="tools_database",
            user="langgraph_user",
            password="langgraph_password"
        )
        conn.autocommit = True

        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        
            # Prepare the INSERT query
            insert_query = """
            INSERT INTO warehouses.inventory 
            (warehouse_id, warehouse_location, warehouse_name, product_id, total_quantity, reserved_quantity)
            VALUES (%(warehouse_id)s, %(warehouse_location)s, %(warehouse_name)s, %(product_id)s, %(total_quantity)s, %(reserved_quantity)s)
            """
            
            # Use execute_batch for better performance with many inserts
            execute_batch(cursor, insert_query, inventory_records, page_size=100)
            
            # Commit the transaction
            conn.commit()
            
            print(f"Successfully inserted {len(inventory_records)} records into warehouses.inventory")
            
            # Close cursor and connection
            cursor.close()
            conn.close()
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [23]:
insert_inventory_to_db(inventory_data)

Successfully inserted 4457 records into warehouses.inventory
